# Data preparation

Here I wanted to prepare the main dataset for usage in creating the model. This dataset will contain the matching players from FM20 and Transfermarkt, including their valuations and change in valuation from 2020 to 2022.

First I imported the libraries and the datasets (FM20, Transfermarkt Players and Transfermarkt valuations over time)

In [2]:
from src.libs import *

fm20_dataset = pd.read_csv("datasets/datafm20.csv")
transfermarkt_players_dataset = pd.read_csv("datasets/players.csv")
transfermarkt_valuations_dataset = pd.read_csv("datasets/player_valuations.csv")

Here I start preparing both datasets to be merged. 

I select only players with less than 27 y.o. This decision was made so we can focus only on players with a high probability of increasing in value (we assume as players get older than this, the probability of a value increase is much smaller).

In [1]:
transfermarkt_base_players_df = transfermarkt_players_dataset[:]
transfermarkt_base_players_df.query("last_season >= 2021 & date_of_birth >= '1992-01-01' & position != 'Goalkeeper'", inplace=True)
transfermarkt_base_players_df.dropna(subset='market_value_in_gbp')

fm20_base_players_df = fm20_dataset.query("Age <= 27 & `Best Pos` != 'GK'")

NameError: name 'transfermarkt_players_dataset' is not defined

To merge both dataframes we need to figure out what will be the criteria for players to "match". We decided to use:
- Player name and nation should always match
- If the name and nation matches for more than one player, the player value will be the tiebreaker (so the highest valued match in one dataset will match in the other dataset) 

So first I ordered both datasets by value. Some players in the transfermarkt dataset didn't have value so I dropped those.

In [ ]:
transfermarkt_base_players_df.dropna(subset='market_value_in_gbp')
transfermarkt_base_players_df.sort_values(by='market_value_in_gbp', ascending=False, inplace=True)

fm20_base_players_df.sort_values(by='Value', ascending=False, inplace=True)

NameError: name 'transfermarkt_players_dataset' is not defined

First I made a script which matched the name in FM with the name which appears as unique in Transfermarkt. This will be used to match players in FM and Transfermarkt.

In [ ]:
fm20_base_players_df['Transfermarkt_Name'] = fm20_base_players_df['Name'].apply(name_char_replacer).apply(name_to_id)

Then, I mapped the country of citizenship in Transfermarkt to the country codes in FM20. This was made by first capturing the 5 most valuable players of each nation, find what is the matching *country_of_citizenship* in Transfermarkt, and then match that to the FM20 *Nation* of those players.

In [4]:

fm20_name_nation_mapping = fm20_base_players_df.groupby('Nation').head(5)[['Name', 'Nation']]

transfermarkt_name_nation_mapping = transfermarkt_players_dataset[['name', 'country_of_citizenship']]

player_nationalities_mapping = fm20_name_nation_mapping.merge(transfermarkt_name_nation_mapping, left_on='Transfermarkt_Name', right_on='name')
player_nationalities_mapping.drop_duplicates(subset='Nation', inplace=True)

fm20_base_players_df = fm20_base_players_df.merge(player_nationalities_mapping[['Nation', 'country_of_citizenship']], on='Nation')

Here I make my first attempt in merging both datasets, using only the name and country of citizenship. I create our **merged dataframe** called *transfermarkt_fm20_merged_df*.

In [6]:
transfermarkt_fm20_merged_df = fm20_base_players_df.merge(transfermarkt_base_players_df, left_on=['Transfermarkt_Name', 'country_of_citizenship'], right_on=['name', 'country_of_citizenship'])

transfermarkt_fm20_merged_df.drop_duplicates(subset=['Transfermarkt_Name', 'country_of_citizenship'], inplace=True)

Here I try to get more results than those I get previously.
- I get all the merged players who are missing from *base_players_df*.
- I sort them by market_value and then figure out their age in 2020 based on the *date_of_birth* (the data I have in Transfermarkt)
    - I can't know an accurate DOB as I only have the age of the player in the FM20 dataframe, and I don't know exactly when the age was "measured"
- Finally, I apply a function which is sort of a fuzzy search taking into account that the name, country, age and market value are within a certain range
    - The name is fuzzy searched with difflib.get_close_matches
    - The value in FM should be greater than or equal to 1/5 of the Transfermarkt value.
    - The age on FM should be equal to the Transfermarkt age or equal to the Transfermarkt age - 1 (due to not knowing exactly if a player already had their birthday or not in FM, the approximate_age_2020 function rounded upwards)

TODO: I would love some tips on performance on this fuzzy search

In [8]:
missing = transfermarkt_base_players_df.merge(transfermarkt_fm20_merged_df, indicator='i', how='outer').query('i == "left_only"')[transfermarkt_base_players_df.columns]

missing.sort_values(by='market_value_in_gbp', ascending=False, inplace=True)
missing['Age_2020'] = missing['date_of_birth'].apply(approximate_age_2020)
missing_fm = fm20_base_players_df[:]

def find_match(player):
    global missing_fm
    country = player['country_of_citizenship']
    age = player['Age_2020']
    value = player['market_value_in_gbp'] / 5
    age_minus = age - 1
    potential_players = missing_fm.query('country_of_citizenship == @country and (Age == @age_minus or Age == @age) and Value >= @value')
    player_name = next(iter(difflib.get_close_matches(player['name'], potential_players['Name'], cutoff=0.5)), None)
    if player_name:
        player_fm = potential_players.query('Name == @player_name').sort_values(by='Value', ascending=False).head(1)
        missing_fm = missing_fm.drop(player_fm.index)
        return player_fm.index
    return None


matches = missing.apply(find_match, axis=1)


Here I append the newly matched values from the missing dataframe values to our **merged dataframe**.

In [9]:
tfmk_missing = missing.merge(matches.to_frame('fm_player_index'), left_index=True, right_index=True).dropna()
tfmk_missing['fm_player_index'] = tfmk_missing['fm_player_index'].apply(lambda x: x.tolist()[0])
tfmk_missing = tfmk_missing.merge(fm20_base_players_df, left_on='fm_player_index', right_index=True)
transfermarkt_fm20_merged_df = transfermarkt_fm20_merged_df.append(tfmk_missing)

Here I insert the target values (valuation now vs valuations 2020) into the **merged dataframe**. This is made using the *transfermarkt_valuations_dataset* and finding the players' values in 2020, and comparing them with their value now using *market_value_in_gbp*. This creates the **final dataframe** called *players_with_values*.

In [11]:

transfermarkt_fm20_merged_df['value_2022'] = transfermarkt_fm20_merged_df['market_value_in_gbp']

player_values_2020 = transfermarkt_valuations_dataset.query("date >= '2019-07-01' & date <= '2020-12-31'").sort_values(by='date', ascending=True).drop_duplicates(subset='player_id')

player_values_2020['value_2020'] = player_values_2020['market_value']

players_with_values = transfermarkt_fm20_merged_df.merge(player_values_2020, on='player_id')

players_with_values['value_diff'] = players_with_values['value_2022'] - players_with_values['value_2020']

Finally, with all the data prepared, I save the **final dataframe** for usage in the predictor.

In [12]:
players_with_values.to_csv('assets/base_players_dataset.csv')